In [15]:
import json
import os

from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI

from azure.identity import DefaultAzureCredential
from sqlalchemy import create_engine

from dotenv import load_dotenv
import os

load_dotenv(override=True)

True

In [16]:
# OpenAI API credentials
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_GPT_DEPLOYMENT = os.getenv("AZURE_OPENAI_TEXT_DEPLOYMENT_NAME","gpt-4o")
AZURE_OPENAI_SERVICE_KEY = os.getenv("AZURE_OPENAI_KEY")


token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    #api_key=os.getenv("AZURE_OPENAI_KEY"),  
    azure_ad_token_provider=token_provider,
    api_version="2025-02-01-preview"
)

max_response_tokens=500

messages=[
                    {"role": "system", "content": "Service desk experience, Hardware performance, Request processes, Resolution times, Other" },
                    {"role": "user", "content": "I always have excellent interactions with IT teams."}
             ]

response = client.chat.completions.create(
    model=AZURE_OPENAI_GPT_DEPLOYMENT,
    messages=messages,
    temperature=0.0,
    max_tokens=max_response_tokens,
    top_p=0.9,
    frequency_penalty=0,
    presence_penalty=0,
)
response.choices[0].message.content

"That's fantastic to hear! Having positive interactions with IT teams can make a huge difference in resolving technical issues and improving overall productivity. It sounds like the IT teams you've worked with are doing a great job in areas like communication, efficiency, and customer service. \n\nIf you'd like, I can provide tips on how to maintain or further enhance these interactions, or share insights into how IT teams typically approach service desk requests, hardware performance, and resolution processes. Let me know!"

In [17]:
system_message = """
You are expert in PostgreSQL queries. Given an input question, create a syntactically correct SQL query AND use the query_db function to execute the query.

 PostgreSQL "recommend_recipe_by_description" table has properties:
    #
    #  in_recipedescription character varying "input recipe description"
    #  out_recipename character varying "similar recipe name"
    #  out_similarityscore real "similarity score between 0 and 1"
    #  

        
Examples:
Question: What recipes are similar to 'chicken and peanut'?

Query:
SELECT in_recipedescription, out_recipename, out_similarityscore 
FROM recommend_recipe_by_description('chicken and peanut', 3)
WHERE out_similarityscore !=0
ORDER BY 2 DESC
; 

Answer:
The top 3 most similar recipes to 'chicken and peanut' are in markdown format: 
markdown table example:
| Recipe Name | Similarity Score |
| ------------- | ----------------- |
| Recipe 1 | 0.95 |
| Recipe 2 | 0.90 |
| Recipe 3 | 0.85 |


    
If the user is asking you for data that is not in the table, you should answer with "Error: <description of the error>":
	
Follow these Instructions:
- Generate a valid SQL query to execute on a postgreSQL database.
- Order results by most similar recipe name.
- **MUST** Call the query_db function to execute the query.
- **MUST** return the results in a markdown table format.

Question: 
"""


In [18]:
# Define the function for the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "query_db",
            "description": "Execute SQL query",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "query to be executed in SQL database",
                    },
                },
                "required": ["query"],
            },
        }
    }
]

In [19]:
messages = [{"role": "system", "content": system_message}]

question = "I am looking for recipe that contains chicken and peanut. Can you suggest me few similar recipes?"        
messages.append({"role": "user", "content": f"{question}"})

response = client.chat.completions.create(
            model= "gpt-4o",
            messages=messages, 
            max_tokens=800,
            temperature=0.0,
            top_p=0.95,
            tools=tools,
            #tool_choice="auto",
            tool_choice={"type": "function", "function": {"name": "query_db"}},
            #seed=78,
        )

response_message = response.choices[0].message.content
print("response_message", response_message)


response_message None


In [20]:
import inspect


# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [21]:
def run_conversation(messages, tools, available_functions):
    # Step 1: send the conversation and available functions to GPT
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        #tool_choice={"type": "function", "function": {"name": "query_db"}},
            #seed=78,
    )

    response_message = response.choices[0].message

    # Step 2: check if GPT wanted to call a function
    if response_message.tool_calls:
        print("Recommended Function call:")
        print(response_message.tool_calls[0])
        print()

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        function_name = response_message.tool_calls[0].function.name

        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]

        # verify function has correct number of arguments
        function_args = json.loads(response_message.tool_calls[0].function.arguments)
        print(f"-------Function arguments:{function_args}")
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = function_to_call(**function_args)

        print("Output of function call:")
        print(function_response)
        print()

        # Step 4: send the info on the function call and function response to GPT

        # adding assistant response to messages
        messages.append(
            {
                "role": response_message.role,
                "function_call": {
                    "name": response_message.tool_calls[0].function.name,
                    "arguments": response_message.tool_calls[0].function.arguments,
                },
                "content": None,
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = client.chat.completions.create(
            messages=messages,
            model="gpt-4o",
        )  # get a new response from GPT where it can see the function response

        return second_response

In [22]:
def query_db(query: str) -> dict:        
        print("Authenticating to Azure Database for PostgreSQL using Azure Identity...")
        POSTGRES_HOST = os.environ["POSTGRES_HOST"]
        POSTGRES_USERNAME = os.environ["POSTGRES_USERNAME"]
        POSTGRES_DATABASE = os.environ["POSTGRES_DATABASE"]
        POSTGRES_SSL = os.environ.get("POSTGRES_SSL")

        azure_credential = DefaultAzureCredential()
        token = azure_credential.get_token("https://ossrdbms-aad.database.windows.net/.default")
        POSTGRES_PASSWORD = token.token

        DATABASE_URI = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}/{POSTGRES_DATABASE}"
        # Specify SSL mode if needed
        DATABASE_URI += f"?sslmode={POSTGRES_SSL}"

        engine = create_engine(DATABASE_URI, echo=False)

        # Run query
        results = []
        with Session(engine) as session:
                most_similars = session.execute(text(query)) #.scalars()
                #print(f"Most similar recipes to 'chicken and peanut':")
                #print(f"--------------------------------------------------")
                for Recipe in most_similars:
                        results.append({"recipe": Recipe[1], "score": Recipe[2]})
                        #print(f"INPUT: {Recipe[0]}: \nOUTPUT: {Recipe[1]} SCORE: ({Recipe[2]})")
                        #print(f"--------------------------------------------------")
        
        
        return json.dumps(results)

In [23]:
available_functions = {
    "query_db": query_db,
}


messages = [{"role": "system", "content": system_message}]

question = "I am looking for recipe that contains chicken and peanut. Can you suggest me few similar recipes?"        
messages.append({"role": "user", "content": f"{question}"})

assistant_response = run_conversation(messages, tools, available_functions)

print("Assistant response final response:")
print(assistant_response.choices[0].message.content)

Recommended Function call:
ChatCompletionMessageToolCall(id='call_eYpgJ3enq9DH2ZG553LMePEc', function=Function(arguments='{"query":"SELECT in_recipedescription, out_recipename, out_similarityscore \\nFROM recommend_recipe_by_description(\'chicken and peanut\', 3)\\nWHERE out_similarityscore !=0\\nORDER BY 2 DESC;"}', name='query_db'), type='function')

-------Function arguments:{'query': "SELECT in_recipedescription, out_recipename, out_similarityscore \nFROM recommend_recipe_by_description('chicken and peanut', 3)\nWHERE out_similarityscore !=0\nORDER BY 2 DESC;"}
Authenticating to Azure Database for PostgreSQL using Azure Identity...
Output of function call:
[{"recipe": "Green Papaya Salad (Som Tam Malakor)", "score": 0.19567707}, {"recipe": "Chicken, Avocado and Mango Salad", "score": 0.19447339}, {"recipe": "Apricot Glazed Chicken", "score": 0.1933773}]

Messages in second request:
{'role': 'system', 'content': '\nYou are expert in PostgreSQL queries. Given an input question, creat